In [ ]:
from bs4 import BeautifulSoup
import os 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
import re
import numpy as np
from scipy.stats import zscore
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from statistics import mode
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import statsmodels.api as sm
import arabic_reshaper
from bidi.algorithm import get_display
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
import pickle
from transformation_functions import square_root_transformation , log_transformation , Reciprocal_transformation




In [ ]:
folder_path = 'C:/Users/IMOE001/Downloads/data/data'
folder_path2 = 'C:/Users/IMOE001/Downloads/data/test' #small sample for testing 

In [ ]:
from bs4 import BeautifulSoup

def extract_features(html_code):
    soup = BeautifulSoup(html_code, 'html.parser') # object of html code 
    features = {}
    
    car_info = soup.find('td', width='65%') # the first matching only 
    if car_info:
        features['اسم السيارة'] = car_info.find('h3').get_text(strip=True)  
        features['موديل السيارة'] = car_info.find('h5').get_text(strip=True)  
    
    car_price = soup.find('h5', class_='post-price')
    if car_price:
        for a_tag in car_price.find_all('a'): # return a list 
            a_tag.extract()
        features['سعر السيارة'] = car_price.get_text(strip=True) if car_price else None 
    
    car_characteristics = soup.find_all('tr', class_='list-row')
    for characteristic in car_characteristics:
        featurename_and_itsvalue = characteristic.find_all('td')
        feature_name = featurename_and_itsvalue[0].get_text(strip=True)
        feature_value = featurename_and_itsvalue[1].get_text(strip=True)
        features[feature_name] = feature_value    
    
    additions = soup.find('td', valign='top', text='إضافات')
    if additions:
        ul_element = additions.find_next('ul')
        li_elements = ul_element.find_all('li')

        list_of_additions = [li.get_text(strip=True) for li in li_elements if li.get_text(strip=True) != 'فتحة سقف']
        print(list_of_additions)
        num_additions = len(list_of_additions)
        features['Number_of_Additions'] = num_additions

        features['فتحة السقف'] = 1 if 'فتحة سقف' in [li.get_text(strip=True) for li in li_elements] else 0


    return features

In [ ]:
all_data = [] 
for filename in os.listdir(folder_path):
    file_path =folder_path  + '/' + filename
    with open(file_path, 'r', encoding='utf-8') as file:
        html_code = file.read()
        data_of_one_sample = extract_features(html_code)
        print(f" Sample: {data_of_one_sample}")
        all_data.append(data_of_one_sample)

#all features => list of dictionarys 

In [ ]:
cars_data = pd.DataFrame(all_data)

In [ ]:
cars_data

In [ ]:
#total_rows_with_missing_values =cars_data.isnull().all(axis=1).sum()
#total_rows_with_missing_values

In [ ]:
csv_file_path = 'C:\\Users\\IMOE001\\Desktop\\cars_data.csv'
cars_data.to_csv(csv_file_path, index=False)

In [ ]:
cars_data = pd.read_csv('cars_data.csv')

In [ ]:
cars_data 

In [ ]:
cars_data = cars_data.drop(columns= 'إضافات')


# 21 features 

In [ ]:
cars_data.shape[1]

# 6988  samples 

In [ ]:
cars_data.shape[0]

In [ ]:
total_rows_with_missing_values = cars_data.isnull().all(axis=1).sum()
total_rows_with_missing_values 

This mean 827 html page dont contain required data about cars 

In [ ]:
total_columns_with_missing_values = cars_data.isnull().all(axis=0).sum()
total_columns_with_missing_values

# drop the rows with missing values 

In [ ]:
cars_data = cars_data.dropna(how='all')


In [ ]:
cars_data.reindex()

In [ ]:
missing_values_in_the_features = cars_data.isnull().sum()
missing_values_in_the_features

# The feature with the most missing values 'الدفع'

In [ ]:
# drop الدفع feature 
cars_data = cars_data.drop(columns='الدفع')


In [ ]:
cars_data

In [ ]:
maping_description = {
    
    'اسم السيارة' : 'Name',
    "لون السيارة": 'color',
    "نوع الوقود": 'fuel_type',
    "أصل السيارة": 'origin_car',
    "عداد السيارة": 'car_speedometer',
    "أصحاب سابقون": 'ex_owners',
    "رخصة السيارة": 'car_license',
    "نوع الجير": 'lime_type',
    "الزجاج": 'glass',
    "قوة الماتور": 'motor_power',
    "عدد الركاب": 'passengers',
    'فتحة السقف': 'sunroof',
    'موديل السيارة' : 'model',
    'سعر السيارة': 'Price',

}

# Create a new DataFrame with updated column names
cars_data= cars_data.rename(columns=maping_description)

# Display the new DataFrame
cars_data

# عداد السيارة , عدد الركاب , اصحاب سابقون 
# The only features with missing values 

In [ ]:
data_types = cars_data.dtypes
data_types

# The numeric features are [motor_power , Number_of_Additions, sunroof]

# The status of the ad does not affect the price of the car

In [ ]:
#Drop features related to ad
ad_columns = ['حالة الإعلان' , 'تاريخ نشر الإعلان' , 'تاريخ إنتهاء الإعلان']

cars_data = cars_data.drop(columns= ad_columns)

In [ ]:

cars_data = cars_data.drop(columns= ['معروضة' , 'وسيلة الدفع'])


In [ ]:
cars_data.reset_index(drop=True)

In [ ]:
cars_data['ex_owners'].unique()

In [ ]:
# Define your mapping dictionary
value_mapping = {
    
    'يد اولى': 1,
    'أولى': 1,
    'يد اولا':1 , 
    'يد أولى' : 1 ,
    'يد أولى استيراد' : 1 , 
    'يد اولي ' : 1 , 
    'واحد' : 1 , 
    'اولا' : 1 ,
    'يد 1' : 1 , 
    '١' : 1 , 
    'يداولى' : 1 , 
    'يد أولى وكالة' : 1 , 
    'يدو اولى': 1 ,
    'يد واله' :1 ,
    'اول':1 , 
    'يد ولا' : 1 , 
    'يد اولى شركة' : 1 , 
    'يد. اولى' : 1 , 
    'ياولى' : 1 , 
    'يد اولى وكالة' : 1 , 
    'يد اولي' : 1 , 
    'يدى اوله' : 1 , 
    'يد اولىً' : 1 , 
    'يد واحد' : 1 , 
    'يد اةلي' : 1 , 
    'يد اولى الماني':1 ,
    'يد  ١' : 1 , 
    '01' : 1 ,
    'يد اولة' : 1 , 
    'ايد أولى' : 1 , 
    'يد اولى استيراد شخصي':1 , 
    'يد ا': 1 ,
    'يد اوله' : 1 , 
    'يداولى سحب شركه':1 , 
    'اولى' : 1 , 
    '1' : 1 ,
    'يد اول' : 1 ,
    'يدأولى' : 1 , 
    'يد' : 1 ,
    'يد ١' : 1 , 
    'يد واحدة' : 1 ,
    'يدأولى' : 1 , 
    'يد اولى (شركة)':1 ,
    'يد اولى اصحاب سابقون 0':1 , 
    'يد اولى  اصل شركة':1 ,
    'وارد كوريا يد اولى' : 1 , 
    'يد أولئ أصحاب سابقون صفر' : 1 , 
    'يد اولى صفر كيلو' : 1 , 
    'اوله'  : 1 , 
    'يداولا' : 1 ,
    '00' : 0 ,
    '0' : 0 , 
    '0000' : 0 , 
    '...' : 0  , 
    '......' : 0 , 
    '0 مستورده' : 0 , 
    '-' : 0 ,
    '00000' : 0 , 
    'استيراد مباشر من كوريا' : 0 ,
    '********' : 0 ,
    '٠' : 0 ,
    '0مستورد' : 0 , 
    '.' : 0 , 
    '.........' : 0 ,
    '00000000' : 0 , 
    '.........' : 0 ,
    '٠٠٠٠٠٠٠٠٠٠' : 0 , 
    'مش منمر . (صفر)' : 0 ,
     '---' : 0 , 
    'Zero' : 0 ,
    '٠٠٠٠٠٠٠' : 0 , 
    'غير منمرمستوردة' : 0 , 
    '..' : 0 , 
    'استيراد كوري' : 0,
    '0 0000000' : 0 ,
    'ستيراد المانيا' : 0 , 
    'استيراد كوريا - جديد مش منمر' : 0 , 
    'ستوردة غير منمرة' : 0 , 
   'استيراد مباشر' : 0 ,
    'اصل شركة' : 0 , 
    'يد أولى،...' : 1 , 
    'Ooo' : 0 , 
    '0000000' : 0 ,
    'يد أولى 1' : 1 , 
    'يد أولي' : 1 , 
   'يد أولئ' : 1 , 
    'يد أولى بعد الاستيراد' : 1 , 
    'ا' : 1 , 
    'مستورده يد اولى' : 1 , 
    'مستوردة غير مرخصة' : 0 ,
    'مستود غير منمر' : 0 , 
    'اصحاب سابقون صفر' : 0 , 
    'استراد شخصي' : 0 , 
    '0 مستورد كوري' : 0 ,
     '******' : 0 ,
    'لا يوجد' : 0 , 
    'مستورد شركة' : 0 , 
    'استيراد'  : 0 , 
    '000000' : 0 , 
    '**********' : 0 ,
   'من الشركه مباشر لا يوجد اصحاب سابقون' : 0 ,
    'استيراد مباشر من اوروبا' : 0 ,
    'ايد اولى' : 1 ,
    'فقط يد اولى' : 1 , 
    '٠٠٠٠' : 0 ,
    'لا شيء' : 0 , 
    'يد 0' : 0 , 
    '،' : 0 , 
    '....' : 0 ,
    '٠٠٠٠٠٠٠٠' : 0 , 
    '٠٠٠٠٠٠٠٠٠' : 0 , 
    '٠٠٠٠٠٠' : 0 ,
    '٠٠' : 0 , 
    '٠٠٠٠٠٠٠٠' : 0 ,
    'سحب شركة' : 0 ,
    'يد اولى، استيراد شخصي' : 1 , 
    'يد اولى فقط' : 1 , 
    'يد أولئ':1 , 
   'مستورده' : 0 , 
    '، 0' : 0 , 
    '٠٠٠٠٠' : 0 ,
    '0استيراد' : 0 ,
    'لايوجد' : 0 ,
   'يد اولى فقط' : 1 , 
    'اولى\x89' : 1 , 
    'يد ولى' : 1 , 
    'مستورد' : 0 ,
    'يد صفر' : 0 ,
    'استيراد الماني' : 0 ,
    '0 استيراد كوري' : 0 , 
    'استيراد المانيا' : 0 , 
    'غير منمر استيراد الماني' : 0 , 
    'صفر' : 0 ,
    '000' : 0 ,
'مستوردة' : 0 ,   
   'استراد' : 0 ,
    'استيراد كوريا': 0 ,
    'وارد المانيا' : 0 ,
    'أستيراد ألماني' : 0 , 
    'مستوردة غير منمرة' : 0 ,
    'O' : 0 , 
    'يد ثانيه' : 2 ,
    'يد ثانية' : 2 ,
    'ثانية':2,
    'يد  ثانية':2, 
    'تانية': 2 ,
    'ايدثانيه':2,
    'تانبه' : 2 ,
    'يد اولئ':1 ,
    'يد تاني':2,
    'ثانبه':2 , 
    'يد تاني':2 ,
    'أيد تانية':2,
    'يد ٢':2,
    'يد ثني': 2,
    'ايد ثانيه':2,
    'ايد ثانية':2,
    'يد ثاني':2 ,
    '2':2 ,
    '٢':2 ,
    'يد 2':2 ,
    'يد اولى ( شركة )': 1 ,
    'يدثانيه':2 ,
    'ثانيه':2,
    'اثنان':2 ,
    'يد تانيه':2,
    'يد ثاتيه':2,
    '2يد':2,
    'ثاني':2,
    'يد2':2,
    'صفرض':0,
    'يدتانيه':2,
    'يد ثاني.':2,
    'صفرر':0,
    'غير منمره':0,
    'غير منمرة':0,
    'مش منمر':0,
    'تاني':2,
    'يد ثالثة':3,
    'ثالثة':3,
    'مش منمرة':0,
    '4':4,
    '3':3,
    'اربعة':4,
    '٣':3,
    '7':7,
    'رابعه':4,
    '٤':4,
    'يد 4':4,
    'يد خامسة':5,    
    'يد رابعه':4,
    'يد ثالثة ( فتيات)':3,
    'يد رابعة':4,
    '5': 5  ,
    '9' : 9 ,
    'يد تالته':3,
    'يد التالته':3,
    'تالته':3,
    'يد خامساً':5,
    'يد 5':5,
    'يد سادسة':6,
    'يد ثالثه':3,
    '6':6,
    'يد ٤ بس نخب السيارة':4,
    'مش منمره':0,
    'يدثالثة':3,
    'يد خامسه':5,
    'يد4':4,
    'يد 3':3,
    'ايد ثالثة':3,
    'ثمانيه':8,
    'يد 6':6,
    'غير منمر':0,
    'يد سابعه':7,
    'يد٢':2,
    'خمسه':5,
    'يد خمسة':5,
    'يد 8':8,
    'يد تانية':2,
    'يدثانية':2,
    'تانيه':2,
    '٨':8,
    '٦':6,
    'رابعة':4,
    'ثالثه':3,
    '8':8,
    '٧':7,
    '٩':9,
    '٥':5,
    'مش منمرا':0,
    'انا':1,
    'يدخامسه':5,
    'يد٤':4,
    'يد ٣':3,
    'يد٥':5,
    'عدد٣':3,
    'يد٣':3,
    'ايد ثالثه':3,
    'تالثه':3,
    'ثالثه':3,
    'يدثالثه':3,
    'يد ثامنه':8,
    'خمس':5,
    'خامسة':5,
    'يد٥':5,
    'يد ٥':5,
    'ثالث':3,
    'يد ٤':4,
    'يدخامسه':5,
    '04':4,
    'يد اربعه':4,
    'يد ٧':7,
    'يديانيه':2,
    'يد6':6,
    '13':13,
    '١٠':10,
    '16':16,
    '17':17,
    '13':13,
    '١١':11,
    '03':3,
    ',3':3,
    '18':18,
    '15':15,
    'عدد٣':3,
    '12':12,
    '11':11,
    '10':10,
    'خمسة':5,
    'يد ثانيا':2,
    '14':14,
    'يد سابعة':7,
    'ايد رابعه':4,
    'خمسة':5,
    '١٨':18,
    'ايد 2':2,
    '١٥':15,
    'يد عاشره':10,
    '10':10,
    '12':12,
    'سابعه':7,
    '14':14,
    'يد ١٧':17,
    'يد سابعة':7,
    'ايد رابعه':4,
    'يد ثالث':3,
    'يد3':3,
    'يد تاسعة':9,
    'خامسه':5,
    '٠٠٠٠٩':9,
    '١٧':17,
    '١٢':12,
    '19':9,
    '١٧':17,
    
    
    
    

    
    
    
  
 'يد اولي 1' : 1

}

cars_data['ex_owners'] =  cars_data['ex_owners'].replace(value_mapping)



In [ ]:
cars_data['ex_owners'].unique()


In [ ]:
cars_data['ex_owners'].value_counts().sort_values()

#  the feature contains noisy data 

In [ ]:
# drop rows Which have noisy
values_to_drop = ['12345', '1000', '؟', '0778244', 'E', '11111', '?', 'مشطوب','170000',  '@', '123423', 'اصل تدريب سواقه', 'اشخاص', 'شطب', '14','159000','نعم','منمره','78888877766','خصوصي','للبيع','56 ',]
cars_data = cars_data[~cars_data['ex_owners'].isin(values_to_drop)]


In [ ]:
cars_data['ex_owners'].isna().sum()

# Filling ex_owners with most occurring Values


In [ ]:
# Filling with most occurring Values

cars_data['ex_owners'].fillna(cars_data['ex_owners'].value_counts().index[0], inplace=True)


In [ ]:
cars_data['ex_owners'].isna().sum()

# convert The Type to int 

In [ ]:
cars_data['ex_owners'] = cars_data['ex_owners'].astype('int64')

In [ ]:
cars_data['ex_owners'].dtype

In [ ]:
def box_plot(The_feature):
    data_column = The_feature
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.boxplot(data_column, vert=False)
    ax.set_xlabel('Feature')
    ax.set_title('Box Plot of Column1')
    plt.show()


In [ ]:
def hist_plot (The_feature) :
    plt.figure(figsize=(8, 6))
    sns.histplot(The_feature, kde=True, color='pink')
    plt.title(f'Distribution of {The_feature}')
    plt.xlabel(feature_to_plot)
    plt.ylabel('Frequency')
    skewness = dr.skew()
    print(f"Skewness: {skewness}")
    plt.show()


In [ ]:
def identify_outliers_iqr(The_feature):
    Q1 = The_feature.quantile(0.25)
    Q3 = The_feature.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return [  lower_bound , upper_bound]
    

In [ ]:
def Remove_the_outliers(data , The_feature , lowest_value , highest_value):
    cleaned_data = data[(The_feature >= lowest_value) & (The_feature <= highest_value)]
    return cleaned_data 

In [ ]:
box_plot(cars_data['ex_owners'])

In [ ]:
identify_outliers_iqr(cars_data['ex_owners'])

# Most cars have only one previous owner, while cars that have more than 5 are the minority

# Remove the outliers 

In [ ]:
cars_data =  Remove_the_outliers(cars_data ,cars_data['ex_owners'] , 0 , 5 ) 

In [ ]:
cars_data['ex_owners'].value_counts().sort_values()

In [ ]:
box_plot(cars_data['ex_owners'])

In [ ]:
cars_data

# motor_power

In [ ]:
cars_data['motor_power'].dtype

# motor power data type is float64

# Covert the data type to int 

In [ ]:
cars_data['motor_power'] = cars_data['motor_power'].astype('int64')

# identifty the outliers 

In [ ]:
identify_outliers_iqr(cars_data['motor_power'])

# the normal range for the 'قوة الماتور ' is between [500,2900] 

In [ ]:
box_plot(cars_data['motor_power'])

# Remove the outliers from motor power

In [ ]:
cars_data = Remove_the_outliers(cars_data ,cars_data['motor_power'], 500 , 3000 ) 

In [ ]:
box_plot(cars_data['motor_power'])

In [ ]:
cars_data

# car_speedometer

# There are numeric values, strings, and symbols in this feature

In [ ]:
 cars_data['car_speedometer'].unique()

In [ ]:
#Trying to standardize the data type 

#delete the strings 
pattern = r'[أ-يا]'
rows_with_letters = cars_data['car_speedometer'].str.contains(pattern, na=False, regex=True)
num_rows_with_letters_before_drop = cars_data[rows_with_letters].shape[0]
num_rows_with_letters_before_drop

In [ ]:
cars_data = cars_data[~rows_with_letters]

In [ ]:
cars_data['car_speedometer'].unique()

In [ ]:
df_copy = cars_data.copy()

In [ ]:
arabic_to_english = {
    '٠': '0',
    '١': '1',
    '٢': '2',
    '٣': '3',
    '٤': '4',
    '٥': '5',
    '٦': '6',
    '٧': '7',
    '٨': '8',
    '٩': '9'
}

def convert_arabic_to_english(arabic_number):
    if pd.notna(arabic_number):
        for arabic, english in arabic_to_english.items():
            arabic_number = arabic_number.replace(arabic, english)
        
        # Remove non-numeric characters
        arabic_number = re.sub(r'[^0-9]', '0' , arabic_number)
        print(arabic_number)

    return arabic_number



In [ ]:
cars_data['car_speedometer'] = cars_data['car_speedometer'].apply(convert_arabic_to_english)


In [ ]:
cars_data['car_speedometer']  = pd.to_numeric(cars_data['car_speedometer'] , errors='coerce')

In [ ]:
cars_data['car_speedometer'].dtype

In [ ]:
 cars_data['car_speedometer'].unique()

In [ ]:
cars_data['car_speedometer'].isna().sum()

# there is a missing value in this feature . 

# Identify outliers 

In [ ]:
lowest , highest = identify_outliers_iqr(cars_data['car_speedometer'])
print(lowest , highest)

In [ ]:
number_of_outliers = cars_data[(cars_data['car_speedometer']  < lowest) | (cars_data['car_speedometer'] > highest)]
len(number_of_outliers)

In [ ]:
#test = cars_data.copy()

# Fill the missing  with median 

In [ ]:
cars_data['car_speedometer'].fillna(cars_data['car_speedometer'].median(), inplace=True)

In [ ]:
Q1 = cars_data['car_speedometer'].quantile(0.25)
Q3 = cars_data['car_speedometer'].quantile(0.85)
IQR = Q3 - Q1
lower_bound_in_speedometer = Q1 - 1.5 * IQR
upper_bound_in_speedometer = Q3 + 1.5 * IQR
lower_bound_in_speedometer , upper_bound_in_speedometer

In [ ]:
cars_data = cars_data[(cars_data['car_speedometer'] > lower_bound_in_speedometer) & (cars_data['car_speedometer'] < upper_bound_in_speedometer ) ]

In [ ]:
cars_data['car_speedometer'] .max()

In [ ]:
box_plot(cars_data['car_speedometer'])

In [ ]:
cars_data.reset_index()

In [ ]:
df = cars_data.copy()

# Now the range of value in the car_speedometer is between [0,333000]

# price


# The data type is object

In [ ]:
cars_data['Price'].dtype

In [ ]:
cars_data['Price'] = cars_data ['Price'].astype(str).str.replace('شيكل', '')



In [ ]:
cars_data.rename(columns={'Price' : 'Price(NIS)'}, inplace=True)

In [ ]:
cars_data['Price(NIS)'] .isna().sum()

# There is no missing value in the price 

In [ ]:
cars_data['Price(NIS)'] = cars_data['Price(NIS)'].astype('int64')

In [ ]:
cars_data['Price(NIS)'].dtype

# Identify the outliers in Price

In [ ]:
box_plot(cars_data['Price(NIS)'])

In [ ]:
lower_bound, upper_bound   = identify_outliers_iqr(cars_data['Price(NIS)'])
lower_bound, upper_bound 

In [ ]:
num_outliers = ((cars_data['Price(NIS)'] < lower_bound) | (cars_data['Price(NIS)'] > upper_bound)).sum()
print(f"Number of outliers in 'Price(NIS)': {num_outliers}")

In [ ]:
 cars_data[cars_data['Price(NIS)'] < 5000][0:50]
#its fine with this prices , the price sutible with modle and speedometer . 

In [ ]:
 cars_data[cars_data['Price(NIS)'] > upper_bound ][0:50]
#Unsuitable or unnatural prices. 

# Remove the outliers 

In [ ]:
cars_data = Remove_the_outliers(cars_data , cars_data['Price(NIS)'] , 0 , 155000.0 ) 

In [ ]:
box_plot(cars_data['Price(NIS)'])

In [ ]:
cars_data['Price(NIS)'].dtype

In [ ]:
cars_data

In [ ]:
cars_data['Price(NIS)'].min()

In [ ]:
cars_data['Price(NIS)'].max()

# Modle feature

In [ ]:
cars_data['model'].dtype

In [ ]:
cars_data['model'].isna().sum()

# There is No missing data 

# Convert the data type 

In [ ]:
cars_data['model'] = cars_data ['model'].astype(str).str.replace('موديل سنة', '')
cars_data['model'] = cars_data['model'].astype('int64')


In [ ]:
cars_data['model'].dtype

In [ ]:
missing_values_in_the_features = cars_data.isnull().sum()
missing_values_in_the_features

In [ ]:
# Filling with most occurring Values
#cars_data = cars_data.apply(lambda x: x.fillna(x.value_counts().index[0]))


In [ ]:
cars_data['model'].value_counts().sort_values()

In [ ]:
box_plot(cars_data['model'])

In [ ]:
 cars_data[cars_data['model'] < 1995][0:50]
#this data can be consider normal data not outliers 

# passengers Feature

In [ ]:
cars_data['passengers'].dtype

In [ ]:
cars_data['passengers'].isna().sum()

# There is missing value in this feature 

In [ ]:
cars_data['passengers'].unique()

# mapping to standrize the format  

In [ ]:
cars_data['passengers'].unique()
noisy_data_in_passengers = []

mapping_for_passengers  =  { 
  '٤+١' : '4+1' ,
  '4+١' : '4+1' , 
  '١+٤' : '4+1' , 
  '1+٤' : '4+1' , 
  '4\\١': '4+1', 
  '7,+ 1': '7+1',
  '7:1' : '7+1' ,
  '7.1' : '7+1' , 
  '٧+1' :  '7+1' , 
  '1+7' : '7+1' , 
  '7 =1': '7+1' , 
  '٧+١' :  '7+1',  
  '6+١' : '6+1' ,
  '6x1' : '6+1' , 
  '٦+١': '6+1' , 
  '٨+١' : '8+1' ,
  'nan' : 'NaN',
  '6±١': '6+1' , 
  '٥+١'  : '5+1' , 
  '4+2' : '5+1' ,
} 
cars_data['passengers'].unique()


In [ ]:
cars_data['passengers'] =  cars_data['passengers'].replace(mapping_for_passengers)

In [ ]:
cars_data['passengers'].unique()

In [ ]:
The_noisy_in_passengers = [ '4+2', '3+1', '1', 'اكثر من 10', '9+1', '1+1' , '٧' ,  '+1', '1+2', '2/1' , '2' , '22']

In [ ]:
cars_data = cars_data[~cars_data['passengers'].isin(The_noisy_in_passengers)]

In [ ]:
cars_data['passengers'].unique()

In [ ]:
cars_data['passengers'].value_counts().sort_values()

In [ ]:
df = cars_data.copy()


# Filling the missing data in passengers using Knn algorithem 

In [ ]:
def impute_non_numeric(neighbors):
    # Extract the 'passengers' values from the neighbors
    passengers_values = neighbors['passengers'].dropna()
    print( 'Hello' , passengers_values)
    # Use mode to find the most common category
    if not passengers_values.empty:
        return mode(passengers_values)
#else all the neighbors , there passengers_values is missing .

   
numeric_features = df[['model', 'Price(NIS)', 'motor_power', 'car_speedometer']]
non_numeric_feature = 'passengers'

missing_indices = df[df[non_numeric_feature].isnull()].index
knn_model = NearestNeighbors(n_neighbors = 5)
knn_model.fit(numeric_features)

for index in missing_indices:
    print(index)
    features_to_impute = numeric_features.loc[index].values.reshape(1, -1)
    print(features_to_impute)
    distances, indices = knn_model.kneighbors(features_to_impute)
    print(distances, indices)
    neighbors = df.iloc[indices[0]]
    print(neighbors['passengers'])
    df.at[index, non_numeric_feature] = impute_non_numeric(neighbors)


In [ ]:
# when k = 1 => 594
# k => 2 =>110
#k => 3 => 28
#k => 5 =>3

In [ ]:
df['passengers'].isna().sum()

In [ ]:
cars_data['passengers'] = df['passengers']

In [ ]:
cars_data= cars_data.dropna(subset=['passengers'])


In [ ]:
cars_data['passengers'].isna().sum()

In [ ]:
len(cars_data['passengers'] )

In [ ]:
cars_data.reset_index()

In [ ]:

def pie_plot(The_feature):
    feature_counts = The_feature.value_counts()
    colors = plt.cm.Paired(range(len(feature_counts)))
    plt.figure(figsize=(20, 10))
    patches, texts, autotexts = plt.pie(feature_counts, labels=feature_counts.index, autopct='',
                                       startangle=140, colors=colors)
    for text, autotext, color in zip(texts, autotexts, colors):
        text.set_color(color)
        autotext.set_color(color)

    plt.title('Pie Chart for Feature')
    plt.rcParams['font.weight'] = 'bold'
    plt.rcParams['font.size'] = 10

    plt.show()


In [ ]:
pie_plot(cars_data['passengers'])

# convert the passengers to int  

In [ ]:
cars_data['passengers'].dtype

In [ ]:
cars_data['passengers'].isna().sum()

In [ ]:
def convert_to_numeric(value):
    if isinstance(value, str):
        parts = value.split('+')
        if len(parts) == 2:
            return int(parts[0]) + int(parts[1])
    return value  # Return the original value for non-string or invalid formats

cars_data['passengers'] = cars_data['passengers'].apply(convert_to_numeric)


In [ ]:
cars_data['passengers'].unique()

In [ ]:
cars_data['passengers'] = cars_data['passengers'].astype('int64')

In [ ]:
missing_values_in_the_features = cars_data.isnull().sum()
missing_values_in_the_features

In [ ]:
cars_data['Number_of_Additions'] = cars_data['Number_of_Additions'].astype('int64')

# Now all features (numeric Features) having no missing values OR outliers 

In [ ]:
cars_data.reset_index()

# Try to normalize and Handle Skewed Distributions

In [ ]:
from_here = cars_data.copy()
from_here 
cars_data = from_here.copy()

In [ ]:
cars_data.kurt()

In [ ]:
cars_data.skew()



# Normlaize model  : -1.452954  ,  passengers :  2.147018 , Price(NIS) : 0.727353 

# Most of Our data is positively skewed => right tail of the distribution 


# But execpt modle and Number_of_Additions  is Negtivaly skewed

In [ ]:
features_to_plot = ['Price(NIS)', 'motor_power', 'car_speedometer', 'passengers', 'ex_owners', 'Number_of_Additions', 'sunroof','model']

fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 8))
fig.suptitle('Distributions of Features')

axes = axes.flatten()

for i, feature in enumerate(features_to_plot):
    sns.histplot(cars_data[feature], kde=True,  color='Purple' ,  ax=axes[i])
    axes[i].set_title(feature)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
def hist_plot (The_feature) : 
    plt.figure(figsize=(8, 6))
    sns.histplot(The_feature, kde=True, color='pink')
    plt.xlabel('price_transformation')
    plt.ylabel('Frequency')
    skewness = The_feature.skew()
    print(f"Skewness: {skewness}")
    plt.show()



In [ ]:
price_log_transformation = log_transformation(cars_data['Price(NIS)'])
hist_plot(price_log_transformation)

In [ ]:
price_Reciprocal_transformation = Reciprocal_transformation(cars_data['Price(NIS)'])
hist_plot(price_Reciprocal_transformation)

In [ ]:
price_square_root_transformation = square_root_transformation(cars_data['Price(NIS)'])
hist_plot(price_square_root_transformation)

In [ ]:
feature_to_plot = price_square_root_transformation
sm.qqplot(feature_to_plot ,  line='s')
plt.show()

# So we choose the square_root transformation 

In [ ]:
root_transformed_data = square_root_transformation(cars_data['Price(NIS)'])
cars_data['Price(NIS)'] = root_transformed_data
transformation_info = {'transformation_function': square_root_transformation, 'transformed_features': root_transformed_data}
with open('root_transformation_info.pkl', 'wb') as file:
    pickle.dump(transformation_info, file)

In [ ]:
cars_data['Price(NIS)'].min()

In [ ]:
cars_data['Price(NIS)'].max()

In [ ]:
Reciprocal_transformed_data = Reciprocal_transformation(cars_data ['passengers'])
cars_data ['passengers'] = Reciprocal_transformed_data
Reciprocal_transformation_info = {'transformation_function': Reciprocal_transformation , 'transformed_features': Reciprocal_transformed_data}
with open('Reciprocal_transformation_info.pkl', 'wb') as file:
    pickle.dump(Reciprocal_transformation_info, file)
   

# The Target Variable is approximately normal.

In [ ]:
#Try to normlize the passengers 

In [ ]:
passengers_log_transformation = log_transformation(cars_data['passengers'])

In [ ]:
hist_plot(passengers_log_transformation)

In [ ]:
passengers_square_root = square_root_transformation(cars_data['passengers'])

In [ ]:
hist_plot(passengers_square_root)

In [ ]:
passengers_Reciprocal_transformation =  Reciprocal_transformation(cars_data['passengers'])
hist_plot(passengers_Reciprocal_transformation)

# tranforme data using Reciprocal_transformation

# value of Kurtosis of passengers : 5.342821 before transformation


In [ ]:
def save_reciprocal_transformation_info(dataframe, feature_name):
    transformed_data = Reciprocal_transformation(dataframe[feature_name])
    transformation_info = {'transformation_function': Reciprocal_transformation, 'transformed_features': transformed_data}
    
    with open(f'{feature_name}_transformation_info.pkl', 'wb') as file:
        pickle.dump(transformation_info, file)

        
Reciprocal_transformed_data = Reciprocal_transformation(cars_data['passengers'])
cars_data['passengers'] = Reciprocal_transformed_data
save_reciprocal_transformation_info(cars_data, 'passengers')


In [ ]:
hist_plot(cars_data['passengers'])

In [ ]:
cars_data['passengers'].kurt() #WOW :) 

In [ ]:
cars_data.skew()

In [ ]:
modle_square_root_transformation = square_root_transformation(cars_data['model'])
hist_plot(modle_square_root_transformation)

In [ ]:
modle_Reciprocal_transformation =  Reciprocal_transformation(cars_data['model'])
hist_plot(modle_Reciprocal_transformation )

In [ ]:
modle_log_transformation = log_transformation(cars_data['model'])
hist_plot(modle_log_transformation)

# NO effect OF Transformation  

# Done of NORMLIZE Features

# Scalling - The numeric data between [0,1] 

In [ ]:
cars_data

In [ ]:
columns_of_interest = ['Price(NIS)', 'model']

# Create a pair plot
sns.pairplot(cars_data[columns_of_interest], height=2.5)
plt.show()

In [ ]:
after_normalized = min_max_normalization(cars_data , ['Price(NIS)', 'model'] )
after_normalized

In [ ]:
columns_of_interest = ['Price(NIS)', 'model']

# Create a pair plot
sns.pairplot(after_normalized[columns_of_interest], height=2.5)
plt.show()

# This means that the Type of correlation after Scalling was not affected  

In [ ]:
def min_max_normalization(dataframe, features_to_normalize):
    if not features_to_normalize:
        # Handle case when features_to_normalize is empty
        return dataframe

    # Separate the 'Price(NIS)' feature from the rest
    price_feature = 'Price(NIS)'
    other_features = [feature for feature in features_to_normalize if feature != price_feature]

    # Create a scaler for 'Price(NIS)'
    scaler_price = MinMaxScaler()
    dataframe[price_feature] = scaler_price.fit_transform(dataframe[price_feature].values.reshape(-1, 1))

    # Create a scaler for the rest of the features
    scaler_other = MinMaxScaler()
    dataframe[other_features] = scaler_other.fit_transform(dataframe[other_features])

    # Save the scalers and features to separate pickle files
    with open('scaler_price.pkl', 'wb') as scaler_file_price:
        pickle.dump({'scaler': scaler_price, 'features': [price_feature]}, scaler_file_price)

    with open('scaler_other.pkl', 'wb') as scaler_file_other:
        pickle.dump({'scaler': scaler_other, 'features': other_features}, scaler_file_other)

    # Save information about the inverse transformation
    inverse_transform_info_price = {'transformation_function': scaler_price.inverse_transform, 'features': [price_feature]}
    inverse_transform_info_other = {'transformation_function': scaler_other.inverse_transform, 'features': other_features}

    with open('inverse_transform_info_price.pkl', 'wb') as inverse_transform_file_price:
        pickle.dump(inverse_transform_info_price, inverse_transform_file_price)

    with open('inverse_transform_info_other.pkl', 'wb') as inverse_transform_file_other:
        pickle.dump(inverse_transform_info_other, inverse_transform_file_other)

    return dataframe



In [ ]:
variables_columns_to_scall = ['Price(NIS)', 'model','motor_power' , 'car_speedometer' , 'passengers' , 'ex_owners' , 'Number_of_Additions' ]
min_max_normalization(cars_data , variables_columns_to_scall )
cars_data

In [ ]:
cars_data

In [ ]:
cars_data['Price(NIS)'].min()

In [ ]:
cars_data['Price(NIS)'].max()

# Now compute The corelation between all numeric features and Target Variable 

In [ ]:
features_to_plot = ['Price(NIS)', 'motor_power', 'car_speedometer', 'passengers', 'ex_owners', 'Number_of_Additions', 'sunroof' ,'model']

# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 8))
fig.suptitle('Scatter Plots and Correlations with Price')

# Flatten the axes for easier indexing
axes = axes.flatten()

for i, feature in enumerate(features_to_plot):
    sns.scatterplot(x=cars_data[feature], y=cars_data['Price(NIS)'], ax=axes[i])
    correlation_coefficient = cars_data[feature].corr(cars_data['Price(NIS)'])
    axes[i].annotate(f'Corr: {correlation_coefficient:.2f}', xy=(0.5, 1.05), xycoords='axes fraction', ha='center', fontsize=10)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


# The strongest relationship is between price and model, which is a positive relationship

# The weakest relationship is between price and the car’s speedometer, and their relationship is negative

# Categorical Feature 

# glass feature

In [ ]:
font_properties = {'font.family': 'Arial', 'font.size': 10, 'font.sans-serif': ['Arial'], 'text.usetex': False}
matplotlib.rcParams.update(font_properties)

features_to_plot = ['fuel_type', 'car_license', 'lime_type', 'glass']

# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
fig.suptitle('Pie Charts of Categorical Features')

# Flatten the axes for easier indexing
axes = axes.flatten()

for i, feature in enumerate(features_to_plot):
    # Count the occurrences of each category in the feature
    feature_counts = cars_data[feature].value_counts()

    # Reshape and reorder Arabic text
    labels = [get_display(arabic_reshaper.reshape(label)) for label in feature_counts.index]

    # Plot a pie chart
    pie = axes[i].pie(feature_counts, labels=labels, autopct='%1.1f%%', startangle=90)
    axes[i].set_title(feature)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


# This   features are imbalanced

In [ ]:
cars_data

In [ ]:
cars_data['origin_car'].unique()


In [ ]:
cars_data['color'].unique()

In [ ]:
categorical_features = ['color', 'origin_car']
fig, axes = plt.subplots(nrows=len(categorical_features), figsize=(10, 6 * len(categorical_features)))
fig.suptitle('Bar Charts of Categorical Features', y=1.02)
for i, feature in enumerate(categorical_features):
    feature_counts = cars_data[feature].value_counts()
    labels = [get_display(arabic_reshaper.reshape(label)) for label in feature_counts.index]
    sns.countplot(x=feature, data=cars_data, ax=axes[i], order=feature_counts.index)
    axes[i].set_title(f'Distribution of {feature}')
    axes[i].set_xticklabels(labels)
plt.tight_layout()
plt.show()


# Convert the  features from nominal to Numeric 

# the features that have two - values use LabelEncoder to convert it 

In [ ]:
def Label_Encoder(The_feature):
    label_encoder = LabelEncoder()
    The_feature_encoded = label_encoder.fit_transform(The_feature)
    return The_feature_encoded

In [ ]:
features_that_have_two_values = ['car_license', 'glass']
for feature in features_that_have_two_values:
    cars_data[feature] = Label_Encoder(cars_data[feature])


In [ ]:
cars_data 

# Now use one-hot encoding to convert feature which contains more than two values (classes)

In [ ]:
def OneHot_Encoder(data, column):
    onehot_encoded_columns = pd.get_dummies(data[column], prefix=f'{column}-')
    data = pd.concat([data, onehot_encoded_columns], axis=1)
    data.drop(column, axis=1, inplace=True)
    return data

In [ ]:
features_have_more_than_two_values = ['Name','color', 'fuel_type','origin_car','lime_type']
for feature in features_have_more_than_two_values:
    cars_data= OneHot_Encoder(cars_data ,feature)


In [ ]:
cars_data.reset_index()


# Now the data is ready to use 

In [ ]:
X = cars_data.drop(['Price(NIS)'],axis=1)  
y = cars_data['Price(NIS)']  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
models = {
    'Linear Regression (Degree 1)': LinearRegression(),
    'Ridge Regression (Degree 1)': Ridge(alpha=1.0),
    'Lasso Regression (Degree 1)': Lasso(alpha=1.0),
    'Polynomial Regression (Degree 2)': make_pipeline(PolynomialFeatures(degree=2), LinearRegression()),
    'Ridge Regression (Degree 2)': make_pipeline(PolynomialFeatures(degree=2), Ridge(alpha=1.0)),
    'Lasso Regression (Degree 2)': make_pipeline(PolynomialFeatures(degree=2), Lasso(alpha=1.0)),
    'K-Nearest Neighbors': KNeighborsRegressor(n_neighbors=3),
    'Decision Tree': DecisionTreeRegressor(random_state=0)
}

results = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    model.fit(X_train, y_train)
    
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    results[model_name] = {
        'MSE_train': mse_train,
        'R2_train': r2_train,
        'MSE_test': mse_test,
        'R2_test': r2_test
    }

for model_name, metrics in results.items():
    print(f"{model_name}:")
    print(f"R-squared (Training): {metrics['R2_train']}")
    print(f"R-squared (Testing): {metrics['R2_test']}")
    
    print(f"Mean Squared Error (Training): {metrics['MSE_train']}")
    print(f"Mean Squared Error (Testing): {metrics['MSE_test']}")


# The Best Model for use is Ridge Regression (Degree 1)


# R-squerd for it 0.8 and MSE = 0.0077

In [ ]:
# Assuming you have the results available in variables like r2_train, r2_test, mse_train, mse_test
results_dict = {
    'Linear Regression (Degree 1)': {
        'R-squared (Training)': 0.8590414261282553,
        'R-squared (Testing)': -4.377152074964761e+19,
        'Mean Squared Error (Training)': 0.004817314454157538,
        'Mean Squared Error (Testing)': 1.635786825066841e+18,
    },
    'Ridge Regression (Degree 1)': {
        'R-squared (Training)': 0.8456910183602393,
        'R-squared (Testing)': 0.7928622692858215,
        'Mean Squared Error (Training)': 0.005273569866959013,
        'Mean Squared Error (Testing)': 0.007740950395908376,
    },
    'Lasso Regression (Degree 1)': {
        'R-squared (Training)': 0.0,
        'R-squared (Testing)': -0.0004996010914921634,
        'Mean Squared Error (Training)': 0.0341753915483049,
        'Mean Squared Error (Testing)': 0.037389700835634525,
    },
    'Polynomial Regression (Degree 2)': {
        'R-squared (Training)': 0.9594533955779442,
        'R-squared (Testing)': -7.196364751207202e+21,
        'Mean Squared Error (Training)': 0.0013856960820779875,
        'Mean Squared Error (Testing)': 2.689355646501025e+20,
    },
    'Ridge Regression (Degree 2)': {
        'R-squared (Training)': 0.9401354600084536,
        'R-squared (Testing)': 0.7974223935561945,
        'Mean Squared Error (Training)': 0.002045894094070254,
        'Mean Squared Error (Testing)': 0.007570533853956182,
    },
    'Lasso Regression (Degree 2)': {
        'R-squared (Training)': 0.0,
        'R-squared (Testing)': -0.0004996010914921634,
        'Mean Squared Error (Training)': 0.0341753915483049,
        'Mean Squared Error (Testing)': 0.037389700835634525,
    },
    'K-Nearest Neighbors': {
        'R-squared (Training)': 0.8119308307004252,
        'R-squared (Testing)': 0.6008887607152252,
        'Mean Squared Error (Training)': 0.006427337498977414,
        'Mean Squared Error (Testing)': 0.014915198187702674,
    },
    'Decision Tree': {
        'R-squared (Training)': 0.999869942636763,
        'R-squared (Testing)': 0.6482851582333176,
        'Mean Squared Error (Training)': 4.444761312365821e-06,
        'Mean Squared Error (Testing)': 0.013143945983348991,
    },
}



In [ ]:
#TO print the result in clear way

In [ ]:
for model_name, metrics in results_dict.items():
    print(model_name + ":")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value}")
    print()

In [ ]:
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
y_pred_train = ridge_model.predict(X_train)
y_pred_test = ridge_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
    
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print(f"R-squared (Training):" ,r2_train )
print(f"R-squared (Testing): " , r2_test)
    
print(f"Mean Squared Error (Training):" ,mse_train )
print(f"Mean Squared Error (Testing):",mse_test)


In [ ]:
with open('ridge_model.pkl', 'wb') as model_file:
    pickle.dump(ridge_model, model_file)